# Testing framework "pipeline"

In [ ]:
import proclam
from proclam import *

In [ ]:
from proclam.simulators import simulator
from proclam.simulators import uniform

In [ ]:
A = proclam.simulators.uniform.Uniform()
truth = A.simulate(3, 100)
# print truth

In [ ]:
from proclam.classifiers import classifier
from proclam.classifiers import guess

In [ ]:
B = proclam.classifiers.guess.Guess()
prediction = B.classify(3, truth)
# print(prediction)

In [ ]:
from proclam.metrics import metric

In [ ]:
C = proclam.metrics.metric.Metric()
performance = C.evaluate(prediction, truth)
# print(performance)